### Importing the necessary dependencies:

In [2]:
import pandas as pd
import numpy as np

In [3]:
from sklearn.utils import resample
from sklearn.preprocessing import MinMaxScaler
from sklearn import metrics
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC

In [4]:
df=pd.read_csv('/home/vboxuser/VoiceFiles/pro_unseg_stat/prosodyfeaturesst_seg_dev_stat.csv')

prosody_train=pd.read_csv('/home/vboxuser/VoiceFiles/prosodyfeaturesst_seg_dev.csv')
prosody_test=pd.read_csv('/home/vboxuser/VoiceFiles/Prosodyfeaturesst_test.csv')

### Preprocessing training data:

In [5]:
prosody_train

,subject,F0avg,F0std,F0max,F0min,F0skew,F0kurt,F0tiltavg,F0mseavg,F0tiltstd,...,kurtosisdurpause,maxdurpause,mindurpause,PVU,PU,UVU,VVU,VP,UP,Group
0,300_P_0.wav,165.908508,38.275139,257.928467,94.493355,0.426894,-0.647107,-65.049128,324.063212,126.268062,...,-0.866593,0.340000,0.149977,0.525115,4.608241,0.113951,0.886049,1.687344,0.217003,HC
1,300_P_1.wav,120.528053,12.529911,146.793304,86.951614,-0.113077,-0.996979,-20.992859,13.360108,24.507299,...,-1.594899,0.600000,0.220000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,HC
2,300_P_10.wav,133.895416,21.495996,205.637115,98.742012,2.033316,3.646668,-149.393324,33.461422,185.119105,...,-0.758323,0.560000,0.320000,0.693961,3.659039,0.189657,0.810343,1.167707,0.273296,HC
3,300_P_11.wav,144.578720,28.399199,238.035019,100.053619,1.236097,0.991367,-15.509460,188.202717,25.538555,...,-2.000000,0.300000,0.190023,0.439975,3.827760,0.114943,0.885057,2.011608,0.261249,HC
4,300_P_12.wav,141.704803,28.684387,214.766144,108.876251,1.277414,0.580108,-18.155109,74.524664,111.242072,...,-1.487127,0.540000,0.150023,0.767724,4.291161,0.178908,0.821092,1.069515,0.233037,HC
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5018,713_P_5.wav,121.557548,36.611835,259.327454,66.125931,1.220141,1.760276,-8.349025,184.875451,123.734211,...,-0.474507,0.549977,0.160000,1.011893,6.071267,0.166669,0.833331,0.823537,0.164710,HC
5019,713_P_6.wav,98.142693,11.498760,151.283951,70.990318,1.337210,3.094978,-107.072079,36.489500,95.307558,...,-1.283715,0.989977,0.289977,1.545837,5.899887,0.262011,0.737989,0.477404,0.169495,HC
5020,713_P_7.wav,112.744049,50.676796,264.040283,67.474419,2.159553,3.558457,-55.952834,24.763987,92.611292,...,0.848076,1.200000,0.169977,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,HC
5021,713_P_8.wav,114.727715,21.992062,206.498825,74.076416,1.794884,4.758493,11.662198,51.707118,276.472244,...,-1.088540,0.520000,0.269977,1.363187,5.240970,0.260102,0.739898,0.542771,0.190804,HC


In [61]:
req_features=df[(df['p_HC_DP']<0.05) | (abs(df['p_HC_DP.1'])>0.4)]
feature_list=req_features['Unnamed: 0'].tolist()

In [62]:
X_train=prosody_train[prosody_train.columns.intersection(feature_list)]
X_train.drop(['maxdurunvoiced','mindurpause'],axis=1,inplace=True)
X_train.head()

/tmp/ipykernel_5861/4214238325.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X_train.drop(['maxdurunvoiced','mindurpause'],axis=1,inplace=True)


,F0min,F0skew,F0kurt,F0tiltavg,F0mseavg,F0tiltstd,F0msestd,F0tiltmax,F0msemax,F0tiltmin,...,skwdurvoiced,maxdurvoiced,mindurvoiced,stddurunvoiced,PVU,PU,UVU,VVU,VP,UP
0,94.493355,0.426894,-0.647107,-65.049128,324.063212,126.268062,620.813173,74.352510,1921.324232,-323.113237,...,0.968927,1.130023,0.089977,0.014787,0.525115,4.608241,0.113951,0.886049,1.687344,0.217003
1,86.951614,-0.113077,-0.996979,-20.992859,13.360108,24.507299,9.612148,13.182021,29.342847,-57.977547,...,1.135397,0.870023,0.190023,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
2,98.742012,2.033316,3.646668,-149.393324,33.461422,185.119105,28.008332,-4.268436,73.582442,-445.876374,...,0.610399,0.969977,0.170023,0.030000,0.693961,3.659039,0.189657,0.810343,1.167707,0.273296
3,100.053619,1.236097,0.991367,-15.509460,188.202717,25.538555,454.583120,10.469975,1390.725713,-79.340829,...,0.236980,0.900000,0.120000,0.036112,0.439975,3.827760,0.114943,0.885057,2.011608,0.261249
4,108.876251,1.277414,0.580108,-18.155109,74.524664,111.242072,126.965636,188.020545,374.338802,-220.148340,...,0.757423,0.700000,0.109977,0.021148,0.767724,4.291161,0.178908,0.821092,1.069515,0.233037


In [63]:
X_train.isnull().sum()

F0min                    0
F0skew                   0
F0kurt                   0
F0tiltavg                0
F0mseavg                 0
F0tiltstd                0
F0msestd                 0
F0tiltmax                0
F0msemax                 0
F0tiltmin                0
F0mseskw                 0
F0tiltku                 0
F0mseku                  0
1F0mean                  0
1F0min                   0
1F0skw                   0
lastF0std                0
lastF0min                0
stdEvoiced               0
avgtiltEvoiced           0
stdtiltEvoiced           0
skwtiltEvoiced           0
kurtosistiltEvoiced      0
skwmseEvoiced            0
avg1Evoiced              0
avglastEvoiced           0
stdlastEvoiced           0
maxlastEvoiced           0
minlastEvoiced           0
avg1Eunvoiced            0
std1Eunvoiced            0
max1Eunvoiced            0
min1Eunvoiced            0
kurtosis1Eunvoiced       0
avglastEunvoiced         0
stdlastEunvoiced         0
maxlastEunvoiced         0
m

#### 1. Checking if the classes have a balanced amount of samples:

In [64]:
#replace string values to 0s & 1s in target column
prosody_train.replace({'Group':{"HC":0,"DP":1}},inplace=True)
Y_train=prosody_train['Group']

In [65]:
val_count=prosody_train.groupby(['Group'])['Group'].count()
print(val_count)

Group
0    3878
1    1145
Name: Group, dtype: int64


In [66]:
! pip install imblearn --quiet

from imblearn.over_sampling import SMOTE

sm = SMOTE(sampling_strategy='minority', random_state=42)

X_train_resampled, Y_train_resampled = sm.fit_resample(X_train,Y_train)
pr_train_upsampled = pd.concat([pd.DataFrame(Y_train_resampled), pd.DataFrame(X_train_resampled)], axis=1)

In [67]:
val_count=pr_train_upsampled.groupby(['Group'])['Group'].count()
print(val_count)

Group
0    3878
1    3878
Name: Group, dtype: int64


#### 2. Obtaining X & Y values:

#### 3. Normalizing the data:

In [68]:
#import Normalization as n

In [69]:
sc=MinMaxScaler()

In [70]:
X=sc.fit_transform(X_train_resampled)

In [ ]:
#X=n.normalize_train(X_train_resampled)
Y=pr_train_upsampled['Group']

In [71]:
print(np.mean(X))
print(np.std(X))

0.3467397077727825
0.23367888810737109


### Preprocessing test data:

In [72]:
prosody_test.head(3)

,subject,F0avg,F0std,F0max,F0min,F0skew,F0kurt,F0tiltavg,F0mseavg,F0tiltstd,...,skwdurpause,kurtosisdurpause,maxdurpause,PVU,PU,UVU,VVU,VP,UP,Group
0,600_AUDIO_patcomb.wav,202.345642,29.188663,332.912964,66.470856,-0.574745,3.575093,-90.397503,124.265354,301.737173,...,2.418669,5.856770,1.100000,1.044438,3.906563,0.267355,0.732645,0.701473,0.255980,HC
1,602_AUDIO_patcomb.wav,153.656647,20.377634,331.162720,66.267441,1.910137,11.193553,-95.629996,71.456519,225.551727,...,3.169202,20.469786,2.610023,1.383277,6.042579,0.228922,0.771078,0.557429,0.165492,DP
2,604_AUDIO_patcomb.wav,124.525162,48.486446,345.921814,65.332237,1.631245,2.545722,-77.961452,54.110385,271.135779,...,2.201472,6.762568,1.950023,1.543144,6.071628,0.254157,0.745843,0.483327,0.164700,DP


In [73]:
prosody_test.isnull().sum()

subject    0
F0avg      0
F0std      0
F0max      0
F0min      0
          ..
UVU        0
VVU        0
VP         0
UP         0
Group      0
Length: 102, dtype: int64

In [74]:
#obtaining X & Y for test data:
prosody_test.replace({'Group':{"HC":0,"DP":1}},inplace=True)
X_test=prosody_test[prosody_test.columns.intersection(feature_list)]
Y_test=prosody_test['Group']

In [75]:
val_count_test=prosody_test.groupby(['Group'])['Group'].count()
print(val_count_test)

Group
0    39
1    17
Name: Group, dtype: int64


In [76]:
#X_test=n.normalize_test(X_test)
X_test=sc.transform(X_test)

In [77]:
print(np.mean(X_test))
print(np.std(X_test))

0.5795976814857178
1.3237008898565714


In [27]:
len(X)

7756

### Finding Best Model:

In [79]:
import model 

In [80]:
obj=model.classification_models()

#### 1. Random Forest Classifier:

In [81]:
obj.RF_model(X,Y,X_test,Y_test)

The RF model accuracy is given as :  0.6964285714285714
              precision    recall  f1-score   support

           0       0.70      1.00      0.82        39
           1       0.00      0.00      0.00        17

    accuracy                           0.70        56
   macro avg       0.35      0.50      0.41        56
weighted avg       0.49      0.70      0.57        56



/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


#### 2. Decision Tree Classifier:

In [82]:
obj.DF_model(X,Y,X_test,Y_test)

The DF model accuracy is given as :  0.5357142857142857
              precision    recall  f1-score   support

           0       0.72      0.54      0.62        39
           1       0.33      0.53      0.41        17

    accuracy                           0.54        56
   macro avg       0.53      0.53      0.51        56
weighted avg       0.61      0.54      0.55        56



#### 3. Logistic Regression Model:

In [83]:
obj.LR_model(X,Y,X_test,Y_test)

The LR_model accuracy is given as :  0.6964285714285714
              precision    recall  f1-score   support

           0       0.70      1.00      0.82        39
           1       0.00      0.00      0.00        17

    accuracy                           0.70        56
   macro avg       0.35      0.50      0.41        56
weighted avg       0.49      0.70      0.57        56



/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


#### 4. Support Vector Machine:

In [84]:
import grid_search as g

ModuleNotFoundError: No module named 'grid_search'

In [ ]:
#parameters=g.best_param(X,Y)

In [85]:
obj.SVM_model(X,Y,10,1,X_test,Y_test)

The SVM_accuracy is given as :  0.6964285714285714
              precision    recall  f1-score   support

           0       0.70      1.00      0.82        39
           1       0.00      0.00      0.00        17

    accuracy                           0.70        56
   macro avg       0.35      0.50      0.41        56
weighted avg       0.49      0.70      0.57        56



/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
